In [7]:
!pip install datasets
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ai4privacy/pii-masking-400k")

Generating dataset pii-masking-400k (C:/Users/Chulpan/.cache/huggingface/datasets/ai4privacy___pii-masking-400k/default/0.0.0/6e80a5852d659186493197b64790257bb60eea22)
Checksum Computation took 0.0 min
Generating train split
Generating train split: 100%|██████████| 325517/325517 [00:04<00:00, 77836.70 examples/s]
Generating validation split
Generating validation split: 100%|██████████| 81379/81379 [00:00<00:00, 134962.66 examples/s]
All the splits matched successfully.
Dataset pii-masking-400k downloaded and prepared to C:/Users/Chulpan/.cache/huggingface/datasets/ai4privacy___pii-masking-400k/default/0.0.0/6e80a5852d659186493197b64790257bb60eea22. Subsequent calls will reuse this data.


In [1]:
!pip install pyarrow pandas

In [4]:
from datasets import load_from_disk
import pandas as pd

# Укажи ПУТЬ К ПАПКЕ, где лежит датасет, а не к самому файлу!
dataset_path = 'pii-masking-400k-validation.arrow' # Например, 'pii-masking-400k-validation'

try:
    # Загружаем весь датасет с диска
    dataset = load_from_disk(dataset_path)

    print("Датасет успешно загружен!")
    print(dataset)

    # Теперь ты можешь работать с ним. Например, посмотреть первый элемент:
    print("\nПервый пример из датасета:")
    print(dataset[0])

    # Если ты хочешь преобразовать его в привычный Pandas DataFrame для анализа:
    # Внимание: это может занять много памяти, если датасет большой!
    # df = dataset.to_pandas()
    # print("\nПервые 5 строк в формате Pandas:")
    # print(df.head())

except FileNotFoundError:
    print(f"Ошибка: Директория не найдена по пути '{dataset_path}'")
    print("Убедись, что указан путь к папке, а не к файлу .arrow")
except Exception as e:
    print(f"Произошла ошибка при загрузке датасета: {e}")

c:\Users\Chulpan\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ошибка: Директория не найдена по пути 'pii-masking-400k-validation.arrow'
Убедись, что указан путь к папке, а не к файлу .arrow


In [5]:
# Файл: 1_prepare_data.py (Финальная, исправленная версия 3.0)

import json
import re
from transformers import AutoTokenizer
from tqdm.auto import tqdm

# --- КОНФИГУРАЦИЯ ---
input_csv_file = "synthetic_logs.csv"
output_jsonl_file = "piiranha_formatted_logs.jsonl"
# Используем тот же токенизатор, что и в оригинальной модели piiranha
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

# --- СЛОВАРЬ МЕТОК ---
TYPE_TO_LABEL = {
    'full_name': 'NAME', 'personName': 'NAME', 'givenName': 'NAME', 'contactName': 'NAME', 'user_fullname': 'NAME',
    'email': 'EMAIL', 'userMail': 'EMAIL', 'contactMail': 'EMAIL', 'eMail': 'EMAIL', 'emailAddr': 'EMAIL', 'mailbox': 'EMAIL',
    'phone': 'TELEPHONENUM', 'contact_phone': 'TELEPHONENUM', 'mobilePhone': 'TELEPHONENUM', 'cellPhone': 'TELEPHONENUM',
    'telephone': 'TELEPHONENUM', 'contactNumber': 'TELEPHONENUM', 'mobileNumber': 'TELEPHONENUM', 'phoneNumber': 'TELEPHONENUM',
    'credit_card': 'CREDITCARDNUMBER', 'card_number': 'CREDITCARDNUMBER', 'bankCard': 'CREDITCARDNUMBER', 'cardNo': 'CREDITCARDNUMBER',
    'cardDetails': 'CREDITCARDNUMBER', 'ccInfo': 'CREDITCARDNUMBER',
    'password': 'PASSWORD', 'pass': 'PASSWORD', 'passphrase': 'PASSWORD', 'pwd': 'PASSWORD', 'loginPass': 'PASSWORD',
    'secret_key': 'PASSWORD', 'credential': 'PASSWORD', 'accessKey': 'PASSWORD',
    'address': 'ADDRESS', 'deliveryAddress': 'ADDRESS', 'home_address': 'ADDRESS', 'physicalAddress': 'ADDRESS',
    'billing_address': 'ADDRESS', 'mailingAddress': 'ADDRESS',
    'ssn': 'SSN', 'nationalID': 'SSN', 'identityNumber': 'SSN', 'socialSecurityNumber': 'SSN', 'ssNumber': 'SSN',
    'tax_id': 'SSN', 'taxIdentification': 'SSN',
    'username': 'USERNAME', 'login': 'USERNAME', 'user_id': 'USERNAME', 'accountName': 'USERNAME', 'userTag': 'USERNAME', 'handle': 'USERNAME',
    'iban': 'IBAN', 'accountID': 'IBAN', 'bankNo': 'IBAN', 'accountNo': 'IBAN', 'bankID': 'IBAN',
    'license_plate': 'PLATENUMBER', 'timestamp': 'DATETIME', 'client_ip': 'IP'
}

def parse_special_csv(file_content):
    """Надежный парсер для сложного CSV."""
    metadata_pattern = re.compile(r',\s*(\d+)\s*,\s*(\d+)\s*,(.*?),\s*(\w+)\s*$', re.MULTILINE)
    header, *rest = file_content.split('\n', 1)
    content = rest[0] if rest else ""
    last_pos = 0
    for match in metadata_pattern.finditer(content):
        log_block = content[last_pos:match.start()].strip()
        if log_block.startswith('"') and log_block.endswith('"'): log_block = log_block[1:-1].replace('""', '"')
        pii_str = match.group(3)
        if pii_str.startswith('"') and pii_str.endswith('"'): pii_str = pii_str[1:-1].replace('""', '"')
        yield {'log_block': log_block, 'pii_info_str': pii_str, 'format_type': match.group(4)}
        last_pos = match.end()

def find_individual_logs(log_block, format_type):
    """Находит отдельные логи внутри блока."""
    if format_type == 'xml':
        return list(re.finditer(r"<log>.*?</log>", log_block, re.DOTALL))
    else: # JSON, CSV, text
        return list(re.finditer(r".+", log_block))

# --- ОСНОВНОЙ СКРИПТ ---
print(f"--- Шаг 2: Обработка файла '{input_csv_file}' ---")
total_records_written = 0
total_pii_found = 0

with open(input_csv_file, 'r', encoding='utf-8') as infile, \
     open(output_jsonl_file, 'w', encoding='utf-8') as outfile:
    
    file_content = infile.read()
    
    for block_data in tqdm(list(parse_special_csv(file_content)), desc="Обработка блоков логов"):
        log_block = block_data['log_block']
        format_type = block_data['format_type']
        
        try:
            pii_info_block = json.loads(block_data['pii_info_str']) if block_data['pii_info_str'] else []
        except json.JSONDecodeError:
            pii_info_block = []

        if not log_block: continue

        for log_match in find_individual_logs(log_block, format_type):
            log_text = log_match.group(0).strip()
            if not log_text: continue
            
            log_start_char, log_end_char = log_match.span()
            
            # --- НОВАЯ, БОЛЕЕ НАДЕЖНАЯ ЛОГИКА РАЗМЕТКИ ---
            # 1. Токенизируем текст и получаем слова (word_ids)
            # ВАЖНО: Мы не можем использовать is_split_into_words=True, т.к. нам нужны оригинальные слова
            tokenized = tokenizer(log_text, return_offsets_mapping=True)
            tokens = tokenizer.convert_ids_to_tokens(tokenized.input_ids)
            word_ids = tokenized.word_ids()

            # 2. Создаем "пустые" метки
            labels = ['O'] * len(tokens)
            
            # 3. Находим PII, относящиеся к этому логу
            for pii in pii_info_block:
                if pii['start'] >= log_start_char and pii['end'] <= log_end_char:
                    # Пересчитываем координаты относительно начала этого лога
                    pii_start_relative = pii['start'] - log_start_char
                    pii_end_relative = pii['end'] - log_start_char
                    pii_type = TYPE_TO_LABEL.get(pii['type'], 'MISC')
                    
                    is_first_token_in_pii = True
                    for i, (start, end) in enumerate(tokenized.offset_mapping):
                        # Проверяем, что токен не является спец. токеном и пересекается с PII
                        if start == end: continue # Пропускаем спец. токены типа [CLS], [SEP]
                        
                        if start < pii_end_relative and end > pii_start_relative:
                            if is_first_token_in_pii:
                                labels[i] = f'B-{pii_type}'
                                is_first_token_in_pii = False
                            else:
                                labels[i] = f'I-{pii_type}'
                    
                    if not is_first_token_in_pii: # Если хоть один токен был помечен
                        total_pii_found += 1

            # 4. Сохраняем результат
            output_record = {
                "source_text": log_text,
                "tokens": tokens,
                "ner_labels": labels,
            }
            outfile.write(json.dumps(output_record, ensure_ascii=False) + '\n')
            total_records_written += 1

print("\n--- Подготовка данных завершена! ---")
print(f"Найдено и обработано PII сущностей: {total_pii_found}")
print(f"Всего создано записей для обучения: {total_records_written}")
print(f"Результат сохранен в: '{output_jsonl_file}'")

--- Шаг 2: Обработка файла 'synthetic_logs.csv' ---


Обработка блоков логов: 100%|██████████| 200/200 [00:00<00:00, 218.70it/s]


--- Подготовка данных завершена! ---
Найдено и обработано PII сущностей: 0
Всего создано записей для обучения: 2417
Результат сохранен в: 'piiranha_formatted_logs.jsonl'


In [9]:
# Файл: prepare_and_tokenize.py (Финальная, рабочая версия)

import json
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset
from tqdm.auto import tqdm

# --- КОНФИГУРАЦИЯ ---
input_csv_file = "synthetic_logs.csv"
tokenized_dataset_path = "./tokenized_log_dataset"
model_name = "iiiorg/piiranha-v1-detect-personal-information"

# --- ЗАГРУЗКА ---
print("--- Шаг 1: Загрузка токенизатора и конфига модели ---")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_config = AutoModelForTokenClassification.from_pretrained(model_name).config
id2label = {int(k): v for k, v in model_config.id2label.items()}
label2id = {v: k for k, v in id2label.items()}

# --- СЛОВАРЬ МЕТОК ---
TYPE_TO_LABEL = {
    'full_name': 'NAME', 'personName': 'NAME', 'givenName': 'NAME', 'contactName': 'NAME', 'user_fullname': 'NAME',
    'email': 'EMAIL', 'userMail': 'EMAIL', 'contactMail': 'EMAIL', 'eMail': 'EMAIL', 'emailAddr': 'EMAIL', 'mailbox': 'EMAIL',
    'phone': 'TELEPHONENUM', 'contact_phone': 'TELEPHONENUM', 'mobilePhone': 'TELEPHONENUM', 'cellPhone': 'TELEPHONENUM',
    'telephone': 'TELEPHONENUM', 'contactNumber': 'TELEPHONENUM', 'mobileNumber': 'TELEPHONENUM', 'phoneNumber': 'TELEPHONENUM',
    'credit_card': 'CREDITCARDNUMBER', 'card_number': 'CREDITCARDNUMBER', 'bankCard': 'CREDITCARDNUMBER', 'cardNo': 'CREDITCARDNUMBER',
    'cardDetails': 'CREDITCARDNUMBER', 'ccInfo': 'CREDITCARDNUMBER',
    'password': 'PASSWORD', 'pass': 'PASSWORD', 'passphrase': 'PASSWORD', 'pwd': 'PASSWORD', 'loginPass': 'PASSWORD',
    'secret_key': 'PASSWORD', 'credential': 'PASSWORD', 'accessKey': 'PASSWORD',
    'address': 'ADDRESS', 'deliveryAddress': 'ADDRESS', 'home_address': 'ADDRESS', 'physicalAddress': 'ADDRESS',
    'billing_address': 'ADDRESS', 'mailingAddress': 'ADDRESS',
    'ssn': 'SSN', 'nationalID': 'SSN', 'identityNumber': 'SSN', 'socialSecurityNumber': 'SSN', 'ssNumber': 'SSN',
    'tax_id': 'SSN', 'taxIdentification': 'SSN',
    'username': 'USERNAME', 'login': 'USERNAME', 'user_id': 'USERNAME', 'accountName': 'USERNAME', 'userTag': 'USERNAME', 'handle': 'USERNAME',
    'iban': 'IBAN', 'accountID': 'IBAN', 'bankNo': 'IBAN', 'accountNo': 'IBAN', 'bankID': 'IBAN',
    'license_plate': 'PLATENUMBER', 'timestamp': 'DATETIME', 'client_ip': 'IP'
}

def parse_special_csv(file_content):
    metadata_pattern = re.compile(r',\s*(\d+)\s*,\s*(\d+)\s*,(.*?),\s*(\w+)\s*$', re.MULTILINE)
    header, *rest = file_content.split('\n', 1)
    content = rest[0] if rest else ""
    last_pos = 0
    for match in metadata_pattern.finditer(content):
        log_block = content[last_pos:match.start()].strip()
        if log_block.startswith('"') and log_block.endswith('"'): log_block = log_block[1:-1].replace('""', '"')
        pii_str = match.group(3)
        if pii_str.startswith('"') and pii_str.endswith('"'): pii_str = pii_str[1:-1].replace('""', '"')
        yield {'log_block': log_block, 'pii_info_str': pii_str, 'format_type': match.group(4)}
        last_pos = match.end()

def find_individual_logs(log_block, format_type):
    if format_type == 'xml':
        return list(re.finditer(r"<log>.*?</log>", log_block, re.DOTALL))
    else:
        return list(re.finditer(r".+", log_block))

# --- ОСНОВНОЙ КОД ОБРАБОТКИ ---
print(f"--- Шаг 2: Обработка файла '{input_csv_file}' ---")
all_records = []
total_pii_found = 0

with open(input_csv_file, 'r', encoding='utf-8') as infile:
    file_content = infile.read()
    
    for block_data in tqdm(list(parse_special_csv(file_content)), desc="Обработка блоков логов"):
        log_block, format_type = block_data['log_block'], block_data['format_type']
        try: pii_info_block = json.loads(block_data['pii_info_str']) if block_data['pii_info_str'] else []
        except json.JSONDecodeError: pii_info_block = []

        if not log_block: continue

        for log_match in find_individual_logs(log_block, format_type):
            log_text = log_match.group(0).strip()
            if not log_text: continue
            
            log_start_char, log_end_char = log_match.span()
            
            # --- САМАЯ НАДЕЖНАЯ ЛОГИКА РАЗМЕТКИ ---
            tokens = tokenizer.tokenize(log_text)
            labels = ['O'] * len(tokens)
            
            pii_in_log = [p for p in pii_info_block if p['start'] >= log_start_char and p['end'] <= log_end_char]
            
            if pii_in_log:
                tokenization = tokenizer(log_text, return_offsets_mapping=True)
                for pii in pii_in_log:
                    pii_start = pii['start'] - log_start_char
                    pii_end = pii['end'] - log_start_char
                    pii_type = TYPE_TO_LABEL.get(pii['type'], 'MISC')
                    
                    is_first = True
                    for i, (start, end) in enumerate(tokenization['offset_mapping']):
                        if start == end: continue # Пропускаем спец. токены
                        if max(start, pii_start) < min(end, pii_end): # Проверка на пересечение
                            if is_first:
                                labels[i-1] = f"B-{pii_type}" # -1 из-за [CLS] токена
                                is_first = False
                            else:
                                labels[i-1] = f"I-{pii_type}"
                    if not is_first:
                        total_pii_found += 1
            
            all_records.append({"tokens": tokens, "ner_tags": labels})

# --- ПРЕОБРАЗОВАНИЕ В ДАТАСЕТ И ТОКЕНИЗАЦИЯ ДЛЯ МОДЕЛИ ---
print("\n--- Шаг 3: Преобразование в формат Hugging Face ---")
dataset = Dataset.from_list(all_records)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=512,
    )
    all_labels = []
    for i, tags in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                tag = tags[word_idx]
                label_ids.append(label2id.get(tag, label2id["O"]))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

print("\n--- Шаг 4: Финальная токенизация и выравнивание меток ---")
tokenized_dataset = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset.column_names,
    desc="Выравнивание меток"
)

# --- СОХРАНЕНИЕ ---
print(f"\n--- Шаг 5: Сохранение готового датасета в '{tokenized_dataset_path}' ---")
tokenized_dataset.save_to_disk(tokenized_dataset_path)

print("\n--- ПОДГОТОВКА ДАННЫХ УСПЕШНО ЗАВЕРШЕНА! ---")
print(f"Найдено и обработано PII сущностей: {total_pii_found}")
print(f"Всего создано записей для обучения: {len(tokenized_dataset)}")
print("Теперь можно запускать скрипт обучения (2_train_model.py)")

--- Шаг 1: Загрузка токенизатора и конфига модели ---
--- Шаг 2: Обработка файла 'synthetic_logs.csv' ---


Обработка блоков логов: 100%|██████████| 200/200 [00:00<00:00, 221.98it/s]



--- Шаг 3: Преобразование в формат Hugging Face ---

--- Шаг 4: Финальная токенизация и выравнивание меток ---


Выравнивание меток: 100%|██████████| 2417/2417 [00:01<00:00, 1552.28 examples/s]



--- Шаг 5: Сохранение готового датасета в './tokenized_log_dataset' ---


Saving the dataset (1/1 shards): 100%|██████████| 2417/2417 [00:00<00:00, 92223.18 examples/s] 


--- ПОДГОТОВКА ДАННЫХ УСПЕШНО ЗАВЕРШЕНА! ---
Найдено и обработано PII сущностей: 0
Всего создано записей для обучения: 2417
Теперь можно запускать скрипт обучения (2_train_model.py)


In [22]:
import json
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset
from tqdm.auto import tqdm
import os
import shutil

# --- КОНФИГУРАЦИЯ ---
input_csv_file = "synthetic_logs.csv"
tokenized_dataset_path = "./tokenized_log_dataset"
model_name = "iiiorg/piiranha-v1-detect-personal-information"

# --- СЛОВАРЬ МЕТОК (самый полный) ---
TYPE_TO_LABEL = {
    'full_name': 'NAME', 'personName': 'NAME', 'givenName': 'NAME', 'contactName': 'NAME', 'user_fullname': 'NAME',
    'email': 'EMAIL', 'userMail': 'EMAIL', 'contactMail': 'EMAIL', 'eMail': 'EMAIL', 'emailAddr': 'EMAIL', 'mailbox': 'EMAIL',
    'phone': 'TELEPHONENUM', 'contact_phone': 'TELEPHONENUM', 'mobilePhone': 'TELEPHONENUM', 'cellPhone': 'TELEPHONENUM',
    'telephone': 'TELEPHONENUM', 'contactNumber': 'TELEPHONENUM', 'mobileNumber': 'TELEPHONENUM', 'phoneNumber': 'TELEPHONENUM', 'tel': 'TELEPHONENUM',
    'credit_card': 'CREDITCARDNUMBER', 'card_number': 'CREDITCARDNUMBER', 'bankCard': 'CREDITCARDNUMBER', 'cardNo': 'CREDITCARDNUMBER',
    'cardDetails': 'CREDITCARDNUMBER', 'ccInfo': 'CREDITCARDNUMBER', 'payment_card': 'CREDITCARDNUMBER', 'paymentCard': 'CREDITCARDNUMBER', 'ccNumber': 'CREDITCARDNUMBER',
    'password': 'PASSWORD', 'pass': 'PASSWORD', 'passphrase': 'PASSWORD', 'pwd': 'PASSWORD', 'loginPass': 'PASSWORD',
    'secret_key': 'PASSWORD', 'credential': 'PASSWORD', 'accessKey': 'PASSWORD', 'secretCode': 'PASSWORD', 'authToken': 'PASSWORD', 'passKey': 'PASSWORD',
    'address': 'ADDRESS', 'deliveryAddress': 'ADDRESS', 'home_address': 'ADDRESS', 'physicalAddress': 'ADDRESS',
    'billing_address': 'ADDRESS', 'mailingAddress': 'ADDRESS', 'location': 'ADDRESS', 'residence': 'ADDRESS', 'street': 'ADDRESS', 'postalAddress': 'ADDRESS',
    'ssn': 'SSN', 'nationalID': 'SSN', 'identityNumber': 'SSN', 'socialSecurityNumber': 'SSN', 'ssNumber': 'SSN',
    'tax_id': 'SSN', 'taxIdentification': 'SSN', 'taxNumber': 'SSN', 'personalID': 'SSN', 'socialID': 'SSN',
    'username': 'USERNAME', 'login': 'USERNAME', 'user_id': 'USERNAME', 'accountName': 'USERNAME', 'userTag': 'USERNAME', 'handle': 'USERNAME',
    'userLogin': 'USERNAME', 'userIdentifier': 'USERNAME', 'nickname': 'USERNAME', 'screenName': 'USERNAME', 'loginID': 'USERNAME', 'userName': 'USERNAME',
    'iban': 'IBAN', 'accountID': 'IBAN', 'bankNo': 'IBAN', 'accountNo': 'IBAN', 'bankID': 'IBAN', 'financialAccount': 'IBAN',
    'account_number': 'IBAN', 'bank_account': 'IBAN', 'accountCode': 'IBAN', 'bankCode': 'IBAN', 'bankAccountNumber': 'IBAN',
    'license_plate': 'PLATENUMBER', 'timestamp': 'DATETIME', 'client_ip': 'IP'
}

print("Ячейка 1: Настройка завершена.")

Ячейка 1: Настройка завершена.


In [28]:
# Ячейка 2 (ИСПРАВЛЕННАЯ)

def parse_special_csv(file_content):
    metadata_pattern = re.compile(r',\s*(\d+)\s*,\s*(\d+)\s*,(.*?),\s*(\w+)\s*$', re.MULTILINE)
    header, *rest = file_content.split('\n', 1)
    content = rest[0] if rest else ""
    last_pos = 0
    for match in metadata_pattern.finditer(content):
        log_block = content[last_pos:match.start()].strip()
        if log_block.startswith('"') and log_block.endswith('"'): log_block = log_block[1:-1].replace('""', '"')
        pii_str = match.group(3)
        if pii_str.startswith('"') and pii_str.endswith('"'): pii_str = pii_str[1:-1].replace('""', '"')
        try:
            pii_info = json.loads(pii_str) if pii_str else []
        except json.JSONDecodeError:
            pii_info = []
        yield {'log_block': log_block, 'pii_info': pii_info}
        last_pos = match.end()

def find_individual_logs(log_block):
    return [line for line in log_block.split('\n') if line.strip()]

print("--- Шаг 2: Чтение и парсинг CSV файла ---")
with open(input_csv_file, 'r', encoding='utf-8') as infile:
    file_content = infile.read()

parsed_data = []
for block_data in tqdm(list(parse_special_csv(file_content)), desc="Обработка блоков"):
    for log_text_raw in find_individual_logs(block_data['log_block']):
        
        # --- КЛЮЧЕВОЕ ИСПРАВЛЕНИЕ ЗДЕСЬ ---
        # Декодируем текст из "кракозябр" (CP1251) в нормальный UTF-8
        try:
            log_text_decoded = log_text_raw.encode('cp1252').decode('cp1251')
        except:
            log_text_decoded = log_text_raw # Если декодирование не удалось, оставляем как есть
        # ------------------------------------

        # Теперь ищем PII в ДЕКОДИРОВАННОМ тексте
        pii_in_log = [p for p in block_data['pii_info'] if p.get('value') and p['value'] in log_text_decoded]
        
        # Сохраняем ДЕКОДИРОВАННЫЙ текст для дальнейшей работы
        parsed_data.append({'log_text': log_text_decoded, 'pii': pii_in_log})

print(f"\nПарсинг завершен. Всего найдено {len(parsed_data)} отдельных логов.")
print("\nПример обработанных данных:")

found_pii_example = False
for record in parsed_data:
    if record['pii']:
        print("--- НАЙДЕН ПРИМЕР С PII! ---")
        print(record)
        found_pii_example = True
        break

if not found_pii_example:
    print("!!! ВНИМАНИЕ: PII все еще не найдены. Проверьте кодировку или формат данных.")
else:
    print("\nОтлично! PII найдены. Можно переходить к следующей ячейке.")

--- Шаг 2: Чтение и парсинг CSV файла ---


Обработка блоков: 100%|██████████| 200/200 [00:00<00:00, 13811.82it/s]


Парсинг завершен. Всего найдено 2417 отдельных логов.

Пример обработанных данных:
!!! ВНИМАНИЕ: PII все еще не найдены. Проверьте кодировку или формат данных.


In [2]:
!pip install evaluate

In [21]:
# Файл: 1_prepare_data.py

from datasets import load_dataset
from transformers import AutoTokenizer

# --- КОНФИГУРАЦИЯ ---
model_name = "iiiorg/piiranha-v1-detect-personal-information"
data_file = "piiranha_formatted_logs.jsonl"
tokenized_dataset_path = "./tokenized_log_dataset" # Куда сохранить готовые данные

# --- ЗАГРУЗКА ---
tokenizer = AutoTokenizer.from_pretrained(model_name)
full_dataset = load_dataset("json", data_files=data_file, split="train")

id2label = {int(k): v for k, v in AutoModelForTokenClassification.from_pretrained(model_name).config.id2label.items()}
label2id = {v: k for k, v in id2label.items()}

# --- ФУНКЦИЯ ТОКЕНИЗАЦИИ ---
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=512,
    )
    all_labels = []
    for i, ner_tags in enumerate(examples["ner_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                tag = ner_tags[word_idx]
                label_ids.append(label2id.get(tag, label2id["O"]))
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

# --- ОБРАБОТКА И СОХРАНЕНИЕ ---
print("Начинаю токенизацию данных...")
tokenized_datasets = full_dataset.map(tokenize_and_align_labels, batched=True)

print(f"Сохраняю обработанный датасет в '{tokenized_dataset_path}'...")
tokenized_datasets.save_to_disk(tokenized_dataset_path)

print("Подготовка данных завершена.")

FileNotFoundError: Unable to find 'D:/Study/studcamp/НГУ/StudcampNGU\piiranha_formatted_logs.jsonl'

In [ ]:
# Файл: 2_train_model.py

from datasets import load_from_disk
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
import evaluate
import numpy as np

# --- КОНФИГУРАЦИЯ ---
model_name = "iiiorg/piiranha-v1-detect-personal-information"
tokenized_dataset_path = "./tokenized_log_dataset"
output_dir = "piiranha-finetuned-logs"

# --- ЗАГРУЗКА ---
print("Загружаю подготовленный датасет...")
tokenized_datasets_from_disk = load_from_disk(tokenized_dataset_path)
dataset_dict = tokenized_datasets_from_disk.train_test_split(test_size=0.1)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

# --- МЕТРИКИ ---
seqeval = evaluate.load("seqeval")
label_list = list(model.config.id2label.values())

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# --- НАСТРОЙКИ ОБУЧЕНИЯ ---
training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# --- ОБУЧЕНИЕ ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Начинаю дообучение модели...")
trainer.train()

# --- СОХРАНЕНИЕ ---
final_model_path = f"{output_dir}-final"
trainer.save_model(final_model_path)
print(f"\nОбучение завершено. Модель сохранена в '{final_model_path}'")

c:\Users\Chulpan\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chulpan\.cache\huggingface\hub\models--iiiorg--piiranha-v1-detect-personal-information. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


KeyboardInterrupt: 